# Data Engineering
The two main tasks in this notebook are:
- Creating the MODELING_RECORDS table
- Generating training data for multiple years

This notebook does not create the MODELING_RECORDS table since the guest use does not have  the appropriate permissions to write to the database.

The code used to create this table is shown for completeness.

We are using the database connection to create CSVs files based on that table.

## Get a network connection
- Import the Python libraries we are using
- Insert a connection from the Data menu

In [ ]:
import pandas as pd
import numpy as np
import ibm_db
import ibm_db_dbi

### Insert the connection credentials in the next cell
 Use the data menu, select connections, select Insert to Code, credentials.
 
 Use the username and password: `user1011`, `u0rl8H#__AAK6?x1`
 
Make sure the empty cell is selected before doing the "insert to code".


In [ ]:
credentials = Db2_guest_connection_credentials

In [ ]:
dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE={0};"
    "HOSTNAME={1};"
    "PORT={2};"
    "PROTOCOL=TCPIP;"
    "UID={3};"
    "PWD={4};").format(credentials['database'], credentials['host'],
                       credentials['port'], credentials['username'],
                       credentials['password'])
schema="HR"

conn = ibm_db.connect(dsn, "", "")
pconn = ibm_db_dbi.Connection(conn)

## Need to process all the years
Do it in two steps:
- For the attrition records where the date range is the 12 months prior to leaving
- For active records where the date range is the current year

You can inspect the next two cells to get an idea on how the data is gathered.

In [ ]:
# Attrition records
attrition_sql = """
SELECT YEAR(E1.TERMINATION_DATE) CURRENT_YEAR,E1.EMPLOYEE_CODE, 
       E3B.POSITION_CODE, E3B.DEPARTMENT_CODE,
       TIMESTAMPDIFF(16,
           CHAR(E1.TERMINATION_DATE - E1.DATE_HIRED)
       ) days_with_company, E1.COMMUTE_TIME,
       TIMESTAMPDIFF(256,
         CHAR((E1.TERMINATION_DATE - 1 YEAR) - E1.BIRTH_DATE)) Age_begin_period,
       E1.GENDER_CODE, 1 as attrition,
       E2.PERIOD_TOTAL_DAYS, E2.STARTING_SALARY,E2.ENDING_SALARY,
       E2.NB_INCREASES, E2.BONUS, E2.NB_BONUS,
       E2.VACATION_DAYS_TAKEN, E2.SICK_DAYS_TAKEN,
       E3.PROMOTIONS,E3.NB_MANAGERS,E3d.DAYS_IN_POSITION,
       E3C.days_since_last_raise,
       E4.RANKING_CODE,E5.OVERTIME,E5.DBLOVERTIME,E5.TRAVEL,
       E6.number_of_classes

     -- EMPLOYEE
FROM {0}.EMPLOYEE as E1,
     
     -- EMPLOYEE_SUMMARY
      (SELECT SUM.EMPLOYEE_CODE, 
              TIMESTAMPDIFF(16,
                CHAR(MAX(SUM.SUMMARY_DATE) - MIN(SUM.SUMMARY_DATE)) ) as Period_Total_days,
              MIN(SUM.SALARY) Starting_salary,
              MAX(SUM.SALARY) Ending_SALARY,
              MAX(SUM.SALARY) - MIN(SUM.SALARY) increase,
              SUM(CASE WHEN SUM.PAY_INCREASE = 0 THEN 0 ELSE 1 END) nb_increases,
              SUM(SUM.BONUS * SUM.SALARY) BONUS,
              SUM(CASE WHEN SUM.BONUS = 0 THEN 0 ELSE 1 END) nb_bonus,
              SUM(SUM.VACATION_DAYS_TAKEN) VACATION_DAYS_TAKEN,
              SUM(SUM.SICK_DAYS_TAKEN) SICK_DAYS_TAKEN
       FROM {0}.EMPLOYEE_SUMMARY SUM, {0}.EMPLOYEE EMP
       WHERE EMP.TERMINATION_CODE = 152
       AND   EMP.EMPLOYEE_CODE = SUM.EMPLOYEE_CODE
       AND   SUMMARY_DATE >= (EMP.TERMINATION_DATE - 1 YEAR)
       AND   SUMMARY_DATE < EMP.TERMINATION_DATE
       GROUP BY SUM.EMPLOYEE_CODE) E2,

     -- EMPLOYEE_HISTORY: number of promotion and managers, days in position
      (SELECT T1.EMPLOYEE_CODE, COUNT(DISTINCT MANAGER_CODE) nb_managers,
       COUNT(DISTINCT POSITION_CODE) - 1 promotions
       FROM (SELECT HST2.EMPLOYEE_CODE, MANAGER_CODE, POSITION_CODE
             FROM {0}.EMPLOYEE_HISTORY HST2, {0}.EMPLOYEE E1
             WHERE HST2.EMPLOYEE_CODE = E1.EMPLOYEE_CODE
             AND   E1.TERMINATION_CODE = 152
             AND   HST2.RECORD_START_DATE = (
                   SELECT MAX(RECORD_START_DATE)
                   FROM   {0}.EMPLOYEE_HISTORY HST3
                   WHERE  HST3.EMPLOYEE_CODE = HST2.EMPLOYEE_CODE)
            UNION ALL 
            SELECT HST.EMPLOYEE_CODE, MANAGER_CODE, POSITION_CODE
            FROM   {0}.EMPLOYEE_HISTORY HST, {0}.EMPLOYEE EMP 
            WHERE  HST.EMPLOYEE_CODE = EMP.EMPLOYEE_CODE
            AND   EMP.TERMINATION_CODE = 152
            AND   HST.RECORD_START_DATE >= (EMP.TERMINATION_DATE - 1 YEAR)
           ) T1
       GROUP BY T1.EMPLOYEE_CODE) E3,
     -- Time in position
      (SELECT H1.EMPLOYEE_CODE, 
        TIMESTAMPDIFF(16,
          CHAR(MIN(EMP.TERMINATION_DATE) - MIN(RECORD_START_DATE)) 
        ) DAYS_IN_POSITION
       FROM   {0}.EMPLOYEE_HISTORY H1, {0}.EMPLOYEE EMP,
              (SELECT HST2.EMPLOYEE_CODE, POSITION_CODE
               FROM {0}.EMPLOYEE_HISTORY HST2, {0}.EMPLOYEE E1
               WHERE HST2.EMPLOYEE_CODE = E1.EMPLOYEE_CODE
               AND   E1.TERMINATION_CODE = 152
               AND   HST2.RECORD_START_DATE = (
                     SELECT MAX(RECORD_START_DATE)
                     FROM   {0}.EMPLOYEE_HISTORY HST3
                     WHERE  HST3.EMPLOYEE_CODE = HST2.EMPLOYEE_CODE)
              ) H2
       WHERE  H1.EMPLOYEE_CODE = EMP.EMPLOYEE_CODE
       AND    H1.EMPLOYEE_CODE = H2.EMPLOYEE_CODE
       AND    EMP.TERMINATION_CODE = 152
       AND    H1.POSITION_CODE = H2.POSITION_CODE
       GROUP BY H1.EMPLOYEE_CODE) E3d,

     -- EMPLOYEE_HISTORY: Latest Department code
      (SELECT HST.EMPLOYEE_CODE, HST.POSITION_CODE,DPT.DEPARTMENT_CODE
          FROM {0}.POSITION_DEPARTMENT DPT,
          (SELECT *
           FROM {0}.EMPLOYEE_HISTORY HST2
           WHERE HST2.RECORD_START_DATE = (
               SELECT MAX(RECORD_START_DATE)
               FROM   {0}.EMPLOYEE_HISTORY HST3, {0}.EMPLOYEE EMP
               WHERE  HST3.EMPLOYEE_CODE = EMP.EMPLOYEE_CODE
               AND    EMP.TERMINATION_CODE = 152
               AND    HST3.RECORD_START_DATE <= TERMINATION_DATE
               AND    HST3.EMPLOYEE_CODE = HST2.EMPLOYEE_CODE)
          ) HST
          WHERE HST.POSITION_CODE = DPT.POSITION_CODE
      ) E3b,

     -- EMPLOYEE_SUMMARY: Days since last raise
     -- Get all Summary up to EMP.TERMINATION_DATE
      (SELECT EMP.EMPLOYEE_CODE, 
       TIMESTAMPDIFF(16,
        CASE WHEN LAST_RAISE_DATE is NULL
        THEN CHAR(LAST_SUMMARY_DATE - DATE_HIRED)
        ELSE CHAR(LAST_SUMMARY_DATE - LAST_RAISE_DATE)
        END ) days_since_last_raise
       FROM {0}.EMPLOYEE EMP,
         (SELECT ESUM.*, ESUM2.LAST_RAISE_DATE FROM
            (SELECT S1.EMPLOYEE_CODE,
                    MAX(SUMMARY_DATE) LAST_SUMMARY_DATE
             FROM   {0}.EMPLOYEE_SUMMARY S1, {0}.EMPLOYEE EMP1
             WHERE S1.EMPLOYEE_CODE = EMP1.EMPLOYEE_CODE
             AND   EMP1.TERMINATION_CODE = 152
             AND   SUMMARY_DATE < EMP1.TERMINATION_DATE
             GROUP BY S1.EMPLOYEE_CODE) ESUM LEFT OUTER JOIN 
            (SELECT S2.EMPLOYEE_CODE,
                    MAX(SUMMARY_DATE) LAST_RAISE_DATE
             FROM   {0}.EMPLOYEE_SUMMARY S2, {0}.EMPLOYEE EMP2
             WHERE S2.EMPLOYEE_CODE = EMP2.EMPLOYEE_CODE
             AND   EMP2.TERMINATION_CODE = 152
             AND   SUMMARY_DATE < EMP2.TERMINATION_DATE
             AND    PAY_INCREASE > 0
             GROUP BY S2.EMPLOYEE_CODE) ESUM2
            ON ESUM.EMPLOYEE_CODE = ESUM2.EMPLOYEE_CODE ) ESUM3
           WHERE EMP.EMPLOYEE_CODE = ESUM3.EMPLOYEE_CODE) E3C,

     -- RANKING_RESULTS
     -- Get the latest ranking
       (SELECT EMP.EMPLOYEE_CODE, RANKING_CODE
        FROM {0}.EMPLOYEE EMP LEFT OUTER JOIN
          (SELECT RK.* 
           FROM {0}.RANKING_RESULTS RK,
             (SELECT TB0.EMPLOYEE_CODE, 
                MAX(TB0.RANKING_DATE) RANKING_DATE
              FROM {0}.RANKING_RESULTS TB0, {0}.EMPLOYEE EMP
              WHERE TB0.EMPLOYEE_CODE = EMP.EMPLOYEE_CODE
              AND   EMP.TERMINATION_CODE = 152
              AND   RANKING_DATE < EMP.TERMINATION_DATE
              GROUP BY TB0.EMPLOYEE_CODE
             ) TB1
           WHERE RK.EMPLOYEE_CODE = TB1.EMPLOYEE_CODE
           AND   RK.RANKING_DATE = TB1.RANKING_DATE) RK1
        ON EMP.EMPLOYEE_CODE = RK1.EMPLOYEE_CODE
        WHERE EMP.TERMINATION_CODE = 152) E4,

     -- EMPLOYEE_EXPENSE_DETAIL
     -- Use all expenses in the time period
       (SELECT EMP.EMPLOYEE_CODE, overtime, dblovertime, travel
        FROM {0}.EMPLOYEE EMP LEFT OUTER JOIN 
          (SELECT EMPLOYEE_CODE, SUM(overtime) overtime,
                  SUM(dblovertime) dblovertime, SUM(travel) travel
           FROM
            (SELECT EXP.EMPLOYEE_CODE,
               EXPENSE_START_DATE EXPENSE_START_DATE,
               CASE WHEN EXP.EXPENSE_TYPE_CODE = 14 -- Overtime
                 THEN 
                 EXP.EXPENSE_UNIT_QUANTITY ELSE 0 END as overtime,
               CASE WHEN EXP.EXPENSE_TYPE_CODE = 15 -- total trip cost
                 THEN 
                 EXP.EXPENSE_UNIT_QUANTITY ELSE 0 END as travel,
               CASE WHEN EXP.EXPENSE_TYPE_CODE = 33 -- total trip cost
                 THEN EXP.EXPENSE_UNIT_QUANTITY ELSE 0 END as dblovertime
             FROM {0}.EMPLOYEE_EXPENSE_DETAIL EXP, {0}.EMPLOYEE EMP    
             WHERE EXP.EMPLOYEE_CODE = EMP.EMPLOYEE_CODE
             AND   EMP.TERMINATION_CODE = 152
             AND   EXP.EXPENSE_START_DATE >= (EMP.TERMINATION_DATE - 1 YEAR)
             AND   EXP.EXPENSE_START_DATE < EMP.TERMINATION_DATE
             AND   EXP.EXPENSE_TYPE_CODE IN (14,15,33) )
             GROUP BY EMPLOYEE_CODE) EXP1
        ON EMP.EMPLOYEE_CODE = EXP1.EMPLOYEE_CODE
        WHERE EMP.TERMINATION_CODE = 152) E5,

       -- TRAINING_DETAILS
       -- Get the training in the current history record
       (SELECT EMP.EMPLOYEE_CODE, number_of_classes
        FROM {0}.EMPLOYEE EMP LEFT OUTER JOIN
          (SELECT TRAIN.EMPLOYEE_CODE, COUNT(*) number_of_classes
           FROM {0}.TRAINING_DETAILS TRAIN, {0}.EMPLOYEE EMP
           WHERE TRAIN.EMPLOYEE_CODE = EMP.EMPLOYEE_CODE
           AND   EMP.TERMINATION_CODE = 152
           AND   TRAIN.TRAINING_DATE >= (EMP.TERMINATION_DATE - 1 YEAR)
           AND   TRAIN.TRAINING_DATE < EMP.TERMINATION_DATE
           GROUP BY TRAIN.EMPLOYEE_CODE) TR1
        ON EMP.EMPLOYEE_CODE = TR1.EMPLOYEE_CODE
        WHERE EMP.TERMINATION_CODE = 152) E6

WHERE E1.TERMINATION_CODE = 152
AND   E1.EMPLOYEE_CODE = E2.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E3.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E3B.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E3C.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E3D.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E4.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E5.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E6.EMPLOYEE_CODE
"""

In [ ]:
mainsql = """
SELECT {1} CURRENT_YEAR,E1.EMPLOYEE_CODE, 
    E3B.POSITION_CODE, E3B.DEPARTMENT_CODE,
       TIMESTAMPDIFF(16,
           CHAR(MAX_DATE - E1.DATE_HIRED)
       ) days_with_company, E1.COMMUTE_TIME,
       TIMESTAMPDIFF(256,
         CHAR('{1}-01-01 00:00:00' - E1.BIRTH_DATE)) Age_begin_period,
       E1.GENDER_CODE,
       -- CASE WHEN E1.TERMINATION_CODE = 152 
       -- THEN 1 ELSE 0 END as attrition,
       0 attrition,
       E2.PERIOD_TOTAL_DAYS, E2.STARTING_SALARY,E2.ENDING_SALARY,
       E2.NB_INCREASES, E2.BONUS, E2.NB_BONUS,
       E2.VACATION_DAYS_TAKEN, E2.SICK_DAYS_TAKEN,
       E3.PROMOTIONS,E3.NB_MANAGERS,E3d.DAYS_IN_POSITION,
       E3C.days_since_last_raise,
       E4.RANKING_CODE,E5.OVERTIME,E5.DBLOVERTIME,E5.TRAVEL,
       E6.number_of_classes

     -- EMPLOYEE
FROM {0}.EMPLOYEE E1,
     
     -- EMPLOYEE_SUMMARY
      (SELECT EMPLOYEE_CODE, MAX(SUMMARY_DATE) MAX_DATE,
              TIMESTAMPDIFF(16,
                CHAR(MAX(SUMMARY_DATE) - MIN(SUMMARY_DATE)) ) as Period_Total_days,
              MIN(SALARY) Starting_salary,
              MAX(SALARY) Ending_SALARY,
              MAX(SALARY) - MIN(SALARY) increase,
              SUM(CASE WHEN PAY_INCREASE = 0 THEN 0 ELSE 1 END) nb_increases,
              SUM(BONUS * SALARY) BONUS,
              SUM(CASE WHEN BONUS = 0 THEN 0 ELSE 1 END) nb_bonus,
              SUM(VACATION_DAYS_TAKEN) VACATION_DAYS_TAKEN,
              SUM(SICK_DAYS_TAKEN) SICK_DAYS_TAKEN
       FROM {0}.EMPLOYEE_SUMMARY
       WHERE SUMMARY_DATE >= '{1}-01-01 00:00:00'
       AND   SUMMARY_DATE < '{2}-01-01 00:00:00'
       GROUP BY EMPLOYEE_CODE) E2,

     -- EMPLOYEE_HISTORY: number of promotion and managers
      (SELECT T1.EMPLOYEE_CODE, COUNT(DISTINCT MANAGER_CODE) nb_managers,
       COUNT(DISTINCT POSITION_CODE) - 1 promotions
       FROM (SELECT HST2.EMPLOYEE_CODE, MANAGER_CODE, POSITION_CODE
             FROM {0}.EMPLOYEE_HISTORY HST2
             WHERE HST2.RECORD_START_DATE = (
                     SELECT MAX(RECORD_START_DATE)
                     FROM   {0}.EMPLOYEE_HISTORY HST3
                     WHERE  HST3.RECORD_START_DATE < '{2}-01-01 00:00:00'
                     AND    HST3.EMPLOYEE_CODE = HST2.EMPLOYEE_CODE)
             UNION ALL 
             SELECT HST.EMPLOYEE_CODE, MANAGER_CODE, POSITION_CODE
             FROM   {0}.EMPLOYEE_HISTORY HST 
             WHERE  HST.RECORD_START_DATE >= '{1}-01-01 00:00:00'
             AND    HST.RECORD_START_DATE < '{2}-01-01 00:00:00'
            ) T1
       GROUP BY T1.EMPLOYEE_CODE) E3,

     -- Days in position
      (SELECT H1.EMPLOYEE_CODE, 
        TIMESTAMPDIFF(16,
          CHAR(MAX(SUMMARY_DATE) - MIN(RECORD_START_DATE)) 
        ) DAYS_IN_POSITION
       FROM   {0}.EMPLOYEE_HISTORY H1, {0}.EMPLOYEE_SUMMARY SUM,
              (SELECT HST2.EMPLOYEE_CODE, POSITION_CODE
               FROM {0}.EMPLOYEE_HISTORY HST2
               WHERE HST2.RECORD_START_DATE = (
                     SELECT MAX(RECORD_START_DATE)
                     FROM   {0}.EMPLOYEE_HISTORY HST3
                     WHERE  HST3.RECORD_START_DATE < '{2}-01-01 00:00:00'
                     AND    HST3.EMPLOYEE_CODE = HST2.EMPLOYEE_CODE)
              ) H2
       WHERE  H1.EMPLOYEE_CODE = H2.EMPLOYEE_CODE
       AND    H1.POSITION_CODE = H2.POSITION_CODE
       AND    SUM.EMPLOYEE_CODE = H1.EMPLOYEE_CODE
       AND    YEAR(SUM.SUMMARY_DATE) = {1}
       GROUP BY H1.EMPLOYEE_CODE) E3d,

     -- EMPLOYEE_HISTORY: Latest Department code
      (SELECT HST.EMPLOYEE_CODE, HST.POSITION_CODE,DPT.DEPARTMENT_CODE
          FROM {0}.POSITION_DEPARTMENT DPT,
          (SELECT *
           FROM {0}.EMPLOYEE_HISTORY HST2
           WHERE HST2.RECORD_START_DATE = (
               SELECT MAX(RECORD_START_DATE)
               FROM   {0}.EMPLOYEE_HISTORY HST3
               WHERE  HST3.RECORD_START_DATE <= '{2}-01-01 00:00:00'
               AND    HST3.EMPLOYEE_CODE = HST2.EMPLOYEE_CODE)
          ) HST
          WHERE HST.POSITION_CODE = DPT.POSITION_CODE
      ) E3b,

     -- EMPLOYEE_SUMMARY: Days since last raise
     -- Get all Summary up to '{2}-01-01 00:00:00'
      (SELECT EMP.EMPLOYEE_CODE, 
         TIMESTAMPDIFF(16,
           CASE WHEN LAST_RAISE_DATE is NULL
           THEN CHAR(LAST_SUMMARY_DATE - DATE_HIRED)
           ELSE CHAR(LAST_SUMMARY_DATE - LAST_RAISE_DATE)
           END 
         ) days_since_last_raise
       FROM {0}.EMPLOYEE EMP,
         (SELECT ESUM.*, ESUM2.LAST_RAISE_DATE FROM
            (SELECT EMPLOYEE_CODE,
                    MAX(SUMMARY_DATE) LAST_SUMMARY_DATE
             FROM   {0}.EMPLOYEE_SUMMARY
             WHERE  SUMMARY_DATE < '{2}-01-01 00:00:00'
             GROUP BY EMPLOYEE_CODE) ESUM LEFT OUTER JOIN 
         (SELECT EMPLOYEE_CODE,
                 MAX(SUMMARY_DATE) LAST_RAISE_DATE
          FROM   {0}.EMPLOYEE_SUMMARY
          WHERE  SUMMARY_DATE < '{2}-01-01 00:00:00'
          AND    PAY_INCREASE > 0
          GROUP BY EMPLOYEE_CODE) ESUM2
          ON ESUM.EMPLOYEE_CODE = ESUM2.EMPLOYEE_CODE ) ESUM3
       WHERE EMP.EMPLOYEE_CODE = ESUM3.EMPLOYEE_CODE) E3C,

     -- RANKING_RESULTS
     -- Get the latest ranking
      (SELECT EMP.EMPLOYEE_CODE, RANKING_CODE
        FROM {0}.EMPLOYEE EMP LEFT OUTER JOIN
          (SELECT RK.* 
           FROM {0}.RANKING_RESULTS RK,
             (SELECT TB0.EMPLOYEE_CODE, 
                MAX(TB0.RANKING_DATE) RANKING_DATE
              FROM {0}.RANKING_RESULTS TB0
              WHERE RANKING_DATE < '{2}-01-01 00:00:00'
              GROUP BY TB0.EMPLOYEE_CODE
             ) TB1
           WHERE RK.EMPLOYEE_CODE = TB1.EMPLOYEE_CODE
           AND   RK.RANKING_DATE = TB1.RANKING_DATE) RK1
        ON EMP.EMPLOYEE_CODE = RK1.EMPLOYEE_CODE
        ) E4,     

     -- EMPLOYEE_EXPENSE_DETAIL
     -- Use all expenses in the time period
       (SELECT EMP.EMPLOYEE_CODE, overtime, dblovertime, travel
        FROM {0}.EMPLOYEE EMP LEFT OUTER JOIN 
          (SELECT EMPLOYEE_CODE, SUM(overtime) overtime,
                  SUM(dblovertime) dblovertime, SUM(travel) travel
           FROM
            (SELECT EXP.EMPLOYEE_CODE,
               EXPENSE_START_DATE EXPENSE_START_DATE,
               CASE WHEN EXP.EXPENSE_TYPE_CODE = 14 -- Overtime
                 THEN 
                 EXP.EXPENSE_UNIT_QUANTITY ELSE 0 END as overtime,
               CASE WHEN EXP.EXPENSE_TYPE_CODE = 15 -- total trip cost
                 THEN 
                 EXP.EXPENSE_UNIT_QUANTITY ELSE 0 END as travel,
               CASE WHEN EXP.EXPENSE_TYPE_CODE = 33 -- total trip cost
                 THEN EXP.EXPENSE_UNIT_QUANTITY ELSE 0 END as dblovertime
             FROM {0}.EMPLOYEE_EXPENSE_DETAIL EXP     
             WHERE EXP.EXPENSE_START_DATE >= '{1}-01-01 00:00:00'
             AND   EXP.EXPENSE_START_DATE < '{2}-01-01 00:00:00'
             AND   EXP.EXPENSE_TYPE_CODE IN (14,15,33) )
             GROUP BY EMPLOYEE_CODE) EXP1
           ON EMP.EMPLOYEE_CODE = EXP1.EMPLOYEE_CODE) E5,

       -- TRAINING_DETAILS
       -- Get the training in the current history record
       (SELECT EMP.EMPLOYEE_CODE, number_of_classes
        FROM {0}.EMPLOYEE EMP LEFT OUTER JOIN
          (SELECT TRAIN.EMPLOYEE_CODE, COUNT(*) number_of_classes
           FROM {0}.TRAINING_DETAILS TRAIN
           WHERE TRAIN.TRAINING_DATE >= '{1}-01-01 00:00:00'
           AND   TRAIN.TRAINING_DATE < '{2}-01-01 00:00:00'
           GROUP BY TRAIN.EMPLOYEE_CODE) TR1
        ON EMP.EMPLOYEE_CODE = TR1.EMPLOYEE_CODE) E6

WHERE (YEAR(E1.TERMINATION_DATE) > {1} OR E1.TERMINATION_CODE = 150)
AND   E1.EMPLOYEE_CODE = E2.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E3.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E3B.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E3C.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E3D.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E4.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E5.EMPLOYEE_CODE
AND   E1.EMPLOYEE_CODE = E6.EMPLOYEE_CODE
"""

### Instead of hardcoding the year to process, get the year from the `EMPLOYEE_SUMMARY` table

In [ ]:
# Get the min and max date
sql = """
  SELECT MIN(SUMMARY_DATE) MIN_DATE, MAX(SUMMARY_DATE) MAX_DATE
  FROM   {0}.EMPLOYEE_SUMMARY;
""".format(schema)

data_pd = pd.read_sql(sql, pconn)
min_year = data_pd.iloc[0]['MIN_DATE'].year
max_year = data_pd.iloc[0]['MAX_DATE'].year
print("Min year: {0}, Max year: {1}".format(min_year,max_year))

### Creating the `MODELING_RECORDS` table
The table already exists. The SQL execution has been commented out.

In [ ]:
# Populate MODELING_RECORDS
createsql = "CREATE TABLE {0}.Modeling_records as ({1}) WITH DATA;"
insertsql = "INSERT INTO {0}.Modeling_records {1};"

cur = pconn.cursor()
ret = 0
# ret = cur.execute("DROP TABLE IF EXISTS {0}.MODELING_RECORDS".format(schema))
print("Drop table returned: {0}".format(ret))
# ret = cur.execute(createsql.format(schema,attrition_sql.format(schema)))
print("Create table with attrition records returned: {0}".format(ret))

for x in range(1 + max_year - min_year) :
    year = min_year + x
    # sql = insertsql.format(schema,mainsql.format(schema,year,year + 1))
    # ret = cur.execute(sql)
    print("{0} processed, return: {1}".format(year, ret))


### Handle Null values
We need to handle the null values for ranking, overtime, and double overtime, travel, and number of classes taken.

Ranking gets a value of 3, others 0

In [ ]:
sqls = [
    "UPDATE {0}.MODELING_RECORDS SET ranking_code = 3 WHERE ranking_code is NULL;",
    "UPDATE {0}.MODELING_RECORDS SET overtime = 0 WHERE overtime is NULL;",
    "UPDATE {0}.MODELING_RECORDS SET dblovertime = 0 WHERE dblovertime is NULL;",
    "UPDATE {0}.MODELING_RECORDS SET travel = 0 WHERE travel is NULL;",
    "UPDATE {0}.MODELING_RECORDS SET number_of_classes = 0 WHERE number_of_classes is NULL;"
]
ret = 0
cur = pconn.cursor()
for sql in sqls :
    # ret = cur.execute(sql.format(schema))
    print("Update returned: {0}".format(ret))

### Create index and update statistics

In [ ]:
sql = "CREATE INDEX model_records_ix ON {0}.MODELING_RECORDS(CURRENT_YEAR, EMPLOYEE_CODE)".format(schema)
cur = pconn.cursor()
# cur.execute(sql)

In [ ]:
table_name = "{0}.MODELING_RECORDS".format(schema)
update_stats = """
CALL SYSPROC.ADMIN_CMD(
'RUNSTATS ON TABLE {0} ON ALL COLUMNS WITH DISTRIBUTION ON ALL COLUMNS AND SAMPLED DETAILED INDEXES ALL SET PROFILE');
""".format(table_name)
    
cur = pconn.cursor()
# cur.execute(update_stats)

## Create CSVs from Modeling table
Use all the attrition records which represents 40% of the records

### Tables modification
The MODELING_RECORDS are already generated. The POSITION_CODE changes were applied to it.<br/>
We need to make sure that the 2020 records selected have a commute time of zero when not a warehouse worker (4800).

In [ ]:
import math

count_sql = """
SELECT COUNT(*) CNT FROM {0}.MODELING_RECORDS
WHERE CURRENT_YEAR <= {1}
AND   ATTRITION = 1;
"""

records_sql = """
SELECT CURRENT_YEAR,EMPLOYEE_CODE,POSITION_CODE,DEPARTMENT_CODE,
       DAYS_WITH_COMPANY,
       CASE WHEN CURRENT_YEAR = 2020 AND POSITION_CODE NOT IN (4700,4800,5800)
       THEN 0 ELSE COMMUTE_TIME END COMMUTE_TIME,
       AGE_BEGIN_PERIOD,GENDER_CODE,ATTRITION,PERIOD_TOTAL_DAYS,
       STARTING_SALARY,ENDING_SALARY,NB_INCREASES,BONUS,
       NB_BONUS,VACATION_DAYS_TAKEN,SICK_DAYS_TAKEN,
       PROMOTIONS,NB_MANAGERS,DAYS_IN_POSITION,
       DAYS_SINCE_LAST_RAISE,RANKING_CODE,OVERTIME,
       DBLOVERTIME,TRAVEL  -- ,NUMBER_OF_CLASSES
FROM {0}.MODELING_RECORDS
WHERE ATTRITION = 1
AND   CURRENT_YEAR <= {1}
UNION ALL
(SELECT CURRENT_YEAR,EMPLOYEE_CODE,POSITION_CODE,DEPARTMENT_CODE,
        DAYS_WITH_COMPANY,
        CASE WHEN CURRENT_YEAR = 2020 AND POSITION_CODE NOT IN (4700,4800,5800)
        THEN 0 ELSE COMMUTE_TIME END COMMUTE_TIME,
        AGE_BEGIN_PERIOD,GENDER_CODE,ATTRITION,PERIOD_TOTAL_DAYS,
        STARTING_SALARY,ENDING_SALARY,NB_INCREASES,BONUS,
        NB_BONUS,VACATION_DAYS_TAKEN,SICK_DAYS_TAKEN,
        PROMOTIONS,NB_MANAGERS,DAYS_IN_POSITION,
        DAYS_SINCE_LAST_RAISE,RANKING_CODE,OVERTIME,
        DBLOVERTIME,TRAVEL -- ,NUMBER_OF_CLASSES 
 FROM {0}.MODELING_RECORDS TABLESAMPLE SYSTEM (20)
WHERE ATTRITION = 0
AND   CURRENT_YEAR <= {1}
AND CURRENT_YEAR > ({1} - 3) -- Current year and previous 2 years
LIMIT {1})
"""

for x in range(1 + max_year - min_year) :
    year = min_year + x
    fname="sample_records_{0}.csv".format(year)
    data_pd = pd.read_sql(count_sql.format(schema,year), pconn)
    cnt = data_pd.iloc[0]['CNT'].astype(int).item()
    cnt2 = math.ceil(cnt * 65 / 35)
    data_pd = pd.read_sql(records_sql.format(schema,year,cnt2),pconn)
    data_pd.to_csv(fname,index=False)
    print("Saves file {0}, {1} rows".format(fname,data_pd.shape[0]) )

In [ ]:
ls -l

### Move the files from the working directory to the project

In [ ]:
# Move the files to the data folder
# Then I need to add them to the project
!mv *.csv /project_data/data_asset

In [ ]:
!ls -l /project_data/data_asset

### Move the data files to the project
At this point the `sample_records*` files are not yet visible to the project.<br/>
They need to be added to the project through the data menu if you want to use them.